In [33]:
using MathOptInterface
using OSQP
using Test
using LinearAlgebra
using DiffOpt

const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities;

In [34]:
# homogeneous quadratic objective
# Min x^2 + xy + y^2 + yz + z^2
# st  x + 2y + 3z >= 4 (c1)
#     x +  y      >= 1 (c2)
#     x, y, z \in R

model = MOI.instantiate(OSQP.Optimizer, with_bridge_type=Float64)
v = MOI.add_variables(model, 3)
@test MOI.get(model, MOI.NumberOfVariables()) == 3

c1 = MOI.add_constraint(
    model,
    MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([-1.0, -2.0, -3.0], v), 0.0),
    MOI.LessThan(-4.0)
)
c2 = MOI.add_constraint(
    model, 
    MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([-1.0, -1.0, 0.0], v), 0.0),
    MOI.LessThan(-1.0)
)

MOI.set(model, MOI.ObjectiveSense(), MOI.MIN_SENSE)
@test MOI.get(model, MOI.ObjectiveSense()) == MOI.MIN_SENSE
obj = MOI.ScalarQuadraticFunction(MOI.ScalarAffineTerm{Float64}[], MOI.ScalarQuadraticTerm.([2.0, 1.0, 2.0, 1.0, 2.0], v[[1,1,2,2,3]], v[[1,2,2,3,3]]), 0.0)
MOI.set(model, MOI.ObjectiveFunction{MOI.ScalarQuadraticFunction{Float64}}(), obj)

MathOptInterface.ScalarQuadraticFunction{Float64}(MathOptInterface.ScalarAffineTerm{Float64}[], MathOptInterface.ScalarQuadraticTerm{Float64}[ScalarQuadraticTerm{Float64}(2.0, VariableIndex(1), VariableIndex(1)), ScalarQuadraticTerm{Float64}(1.0, VariableIndex(1), VariableIndex(2)), ScalarQuadraticTerm{Float64}(2.0, VariableIndex(2), VariableIndex(2)), ScalarQuadraticTerm{Float64}(1.0, VariableIndex(2), VariableIndex(3)), ScalarQuadraticTerm{Float64}(2.0, VariableIndex(3), VariableIndex(3))], 0.0)

In [35]:
dm = diff_model(model)

#1 (generic function with 1 method)

In [36]:
z, λ, ν = dm.forward()

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 3, constraints m = 2
          nnz(P) + nnz(A) = 11
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   4.00e+00   4.66e-01   1.00e-01   9.99e-05s
  50   1.8571e+00   1.08e-11   6.31e-12   3.96e+00   1.29e-04s

status:               solved
number of iterations

([0.571429, 0.428571, 0.857143], [-0.714286, -0.857143], Float64[])

In [37]:
@test z ≈ [4/7, 3/7, 6/7] atol=1e-4 rtol=1e-4

Test Passed

In [ ]:
# @test dz_dQ ≈ [-0.12244895  0.01530609  -0.11224488;
#                0.01530609   0.09183674   0.07653058;
#                -0.11224488  0.07653058  -0.06122449] atol=1e-4 rtol=1e-4

In [38]:
nz = 3
nineq = 2

Q = [2. 1. 0.; 1. 2. 1.; 0. 1. 2.]
q = [0.; 0.; 0]
G = [-1. -2. -3.; -1. -1. 0.]
h = [-4.; -1.]

2-element Array{Float64,1}:
 -4.0
 -1.0

In [39]:
# lhs = DiffOpt.create_LHS_matrix(z, λ, Q, G, h)
lhs = [Q    G'*Diagonal(λ);
       G    Diagonal((G * z) - h)]
lhs = Matrix(lhs)

5×5 Array{Float64,2}:
  2.0   1.0   0.0   0.714286      0.857143   
  1.0   2.0   1.0   1.42857       0.857143   
  0.0   1.0   2.0   2.14286       0.0        
 -1.0  -2.0  -3.0  -1.07967e-11   0.0        
 -1.0  -1.0   0.0   0.0          -3.18412e-12

In [54]:
# rhs = [[1.1; 0.9; 0.8];
#         zeros(2,1)]


rhs = [ones(3,1);  # setting dl/dz = [1, 1, ...]
        zeros(2,1)]

5×1 Array{Float64,2}:
 1.0
 1.0
 1.0
 0.0
 0.0

In [55]:
sol = -(lhs \ rhs)

5×1 Array{Float64,2}:
 -0.21428571428487383
  0.21428571428646592
 -0.07142857142755321
 -0.5000000000012236 
 -0.5000000000016235 

In [56]:
dz = sol[1:3]
dλ = sol[4:5]

2-element Array{Float64,1}:
 -0.5000000000012236
 -0.5000000000016235

In [57]:
dl_dQ = 0.5*(dz*z' + z*dz')

3×3 Array{Float64,2}:
 -0.122449   0.0153061  -0.112245 
  0.0153061  0.0918367   0.0765306
 -0.112245   0.0765306  -0.0612245

In [58]:
dl_dq = dz

3-element Array{Float64,1}:
 -0.21428571428487383
  0.21428571428646592
 -0.07142857142755321

In [59]:
dl_dG = Diagonal(λ)*dλ*z' - λ*dz' 

2×3 Array{Float64,2}:
 0.0510204  0.306122  0.255102
 0.0612245  0.367347  0.306122

In [60]:
dl_dh = -Diagonal(λ) * dλ

2-element Array{Float64,1}:
 -0.3571428571437865 
 -0.42857142857293173